In [1]:
import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

### Setup

In [3]:
years = []
holidays = {}
week_numbers = []
week_dummies = {}
month_numbers = []
month_dummies = {}

country_id = "GB"
week_ending = False
day = "SAT"
uk_country = "NIR"
country = country_id + "-" + uk_country
start_date = datetime.strptime("01/01/2018", r"%d/%m/%Y").strftime(r"%Y-%m-%d")
end_date = datetime.strptime("30/06/2023", r"%d/%m/%Y").strftime(r"%Y-%m-%d")

start_year = int(datetime.strptime(start_date, r"%Y-%m-%d").strftime(r"%Y"))
end_year = int(datetime.strptime(end_date, r"%Y-%m-%d").strftime(r"%Y"))

while start_year <= end_year:
    years.append(start_year)
    start_year += 1

In [5]:
uri = f"https://date.nager.at/api/v3/PublicHolidays/2023/IE"
connection = requests.get(uri)
response = json.loads(connection.content)
for item in response:
    print(item["localName"], " - ",  item["name"])

Lá Caille  -  New Year's Day
Lá Fhéile Bríde  -  Saint Brigid's Day
Lá Fhéile Pádraig  -  Saint Patrick's Day
Aoine an Chéasta  -  Good Friday
Luan Cásca  -  Easter Monday
Lá Bealtaine  -  May Day
Lá Saoire i mí an Mheithimh  -  June Holiday
Lá Saoire i mí Lúnasa  -  August Holiday
Lá Saoire i mí Dheireadh Fómhair  -  October Holiday
Lá Nollag  -  Christmas Day
Lá Fhéile Stiofáin  -  St. Stephen's Day


## Code

In [ ]:
dates = pd.date_range(start= start_date, end= end_date, freq= "D")
df = pd.DataFrame({"date": dates})
df.head()

In [ ]:
for year in years:
    uri = f"https://date.nager.at/api/v3/PublicHolidays/{year}/{country_id}"
    connection = requests.get(uri)
    response = json.loads(connection.content)
    
    # Parses the API's response
    for item in response:
        try:
            if item["types"][0] == "Public" and country in item["counties"]:
                # Extracts the name of the bank holiday and the date
                holiday_name = item["localName"]
                holiday_date = pd.to_datetime(item["date"], format= r"%Y-%m-%d")
                
                # Creates the (key, value) pair
                if holiday_name not in holidays:
                    holidays[holiday_name] = list()
                    holidays[holiday_name].append(holiday_date)
                else:
                    holidays[holiday_name].append(holiday_date)
        except:
            if item["types"][0] == "Public" and item["counties"] == None:
                # Extracts the name of the bank holiday and the date
                holiday_name = item["localName"]
                holiday_date = pd.to_datetime(item["date"], format= r"%Y-%m-%d")
                
                # Creates the (key, value) pair
                if holiday_name not in holidays:
                    holidays[holiday_name] = list()
                    holidays[holiday_name].append(holiday_date)
                else:
                    holidays[holiday_name].append(holiday_date)
    
for holiday in holidays:
    new_column = list()

    # Creates a list containing 1s and 0s
    for (index, row) in df.iterrows():
        if row["date"] in holidays[holiday][:]:
            new_column.append(1)
        else:
            new_column.append(0)

    # Creates the bank holiday column
    df[holiday + " BH"] = new_column

df = df.resample(f"W-{day}", label= "right", on= "date").sum()

df.reset_index(inplace= True)
df["date"] = df["date"].dt.strftime(r"%d/%m/%Y")

# Solves punctuation issues
df = df.rename(columns= {'''Queen’s Platinum Jubilee BH''': '''Queen's Platinum Jubilee BH''', '''Queen’s State Funeral BH''': '''Queen's State Funeral BH''',
                        "Early May Bank Holiday BH": "Early May BH", "Spring Bank Holiday BH": "Spring BH", "Coronation Bank Holiday BH": "Coronation BH"})

df.head()

In [ ]:
df["account"] = "national"
columns = df.columns.tolist()
df.head()

In [ ]:
columns = columns[-1:] + columns[:-1]
df = df[columns]
df.head()